In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets seqeval tokenizers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'MY-API-KEY'

In [ ]:
!git clone https://github.com/chriswu99aaa/MeTNet.git

Cloning into 'MeTNet'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 241 (delta 6), reused 0 (delta 0), pack-reused 228
Receiving objects: 100% (241/241), 32.25 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (128/128), done.


## Loading Data and Preprocessing

In [ ]:
from pathlib import Path
import re

def read_file(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            if '\t' in line:
                token, tag = line.split('\t')
                tokens.append(token)
                tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [ ]:
train_texts, train_tags = read_file('/content/MeTNet/data/Few-COMM/train.txt')

texts, tags = read_file('/content/MeTNet/data/Few-COMM/dev.txt')

test_text, test_tags = read_file('/content/MeTNet/data/Few-COMM/test.txt')

In [ ]:
query_texts = texts[:200][:]
query_tags = tags[:200][:]

In [ ]:
# get tags from tow datasets and union the two sets
unique_tags = set(tag for doc in train_tags for tag in doc)
unique_tags_val = set(tag for doc in tags for tag in doc)
unique_tags = unique_tags | unique_tags_val

tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
tag2id['B-冲泡方式']

119

In [ ]:
id2tag[5]

'I-色系'

In [ ]:
query = query_texts[0]
query_tag = query_tags[0]

In [ ]:
query_tag

['B-适用时间',
 'I-适用时间',
 'I-适用时间',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
query

['秋',
 '冬',
 '季',
 '新',
 '款',
 '保',
 '暖',
 '棉',
 '拖',
 '鞋',
 '居',
 '家',
 '室',
 '内',
 '踩',
 '屎',
 '感',
 '静',
 '音',
 '厚',
 '底',
 '简',
 '约',
 '防',
 '滑',
 '情',
 '侣',
 '拖',
 '鞋']

In [ ]:
query = ' '.join(query)

In [ ]:
import re
example_query = """维	O
他	O
和	O
梨	O
饮	O
料	O
2	O
5	O
0	O
m	O
l	O
*	O
6	O
盒	O
/	O
组	O
（	O
香	B-原产地
港	I-原产地
进	I-原产地
口	I-原产地
）	O
"""

## Constructing Prompt

In [ ]:
system_prompt = f"""
    You are a Chinese Named Entity Recognizer. The input data is in Chinese and the
    data is annotated in BIO scheme.
    The full list of BIO Tags are provided in the following part delimeted by triple #
    ###
    'B-国产/进口','B-适用人群','I-适用人群','B-适用空间','I-版型','B-面料材质','B-产地','B-酸碱度',
    'B-果肉颜色','B-锅底类型','B-是否去骨','I-适用空间','I-是否有机','B-外观','B-袖长','I-长短','B-组合形式', 'I-送礼对象',
    'I-袖型','I-型号','B-型号','B-形状形态','I-接口','I-是否净洗','I-供电方式','B-送礼对象','B-是否去皮','B-是否净洗','I-适用人数',
    'I-定制服务','B-定制服务','I-适用对象','B-供电方式','I-产地','I-配件类型','I-脂肪含量','I-大小','B-其他属性','O','B-厚薄',
    'I-糖含量','I-面料材质','I-袖长','I-造型','I-颜色','I-是否去骨','B-赠品','I-其他属性', 'I-品质等级','I-适用运营商','I-组合形式',
    'I-果肉颜色','B-接口','I-功能功效','B-适用人数','B-版型','B-材质','I-厚度','B-功能功效','B-长短' 'B-造型','I-成分','I-填充材质',
    'I-是否去皮','B-适用衣物','I-国产/进口','I-鞋垫材质','B-填充材质', 'B-运输服务',
    'I-控制方式', 'I-适用衣物','B-品质等级','B-适用运营商','B-适用对象','I-存储容量','I-香型','I-运输服务','I-商品特色',
    'B-脂肪含量','I-厚薄','B-控制方式','B-筒高','I-长度','I-外观','B-领型','B-分类','I-适用季节','B-适用季节','B-糖含量','B-存储容量',
    'I-分类','B-商品特色','I-赠品','B-香型','B-是否有机','I-材质','I-酸碱度','B-厚度','B-礼盒类型','I-筒高','B-袖型','I-形状形态',
    'B-成分','B-大小','I-领型','B-鞋垫材质','B-长度', 'I-锅底类型', 'B-配件类型', 'B-颜色',
    'I-礼盒类型'
    ###
'''
    When giving a input setence in chinese classify each character
    using the following format
    character BIOtag
    keep numbers and symbols in the output
"""

In [ ]:
num_query_tags = len(unique_tags)

In [ ]:
text_mapping = {}
assigned = [0]*num_query_tags

In [ ]:
# sampling each class with one entity
test_tags = query_tags[:8000]
for i in range(len(test_tags)):
    tag = test_tags[i]
    for j in range(len(tag)):
        if tag[j] != 'O':
            # the index corresponds to the id of the tag
            id = tag2id[tag[j]]
            word = query_texts[i][j]
            if assigned[id] == 0:
                # give one example for each class
                text_mapping[word] = tag[j]
                assigned[id] = 1
                print(text_mapping[word])
                print(id)

In [ ]:
text_mapping = str(text_mapping).replace(':','')

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
  ]
)

print(completion.choices[0].message)



ChatCompletionMessage(content='粉 O\n金 O\n粉 O\n玫 O\n瑰 O\n香 O\n槟 O\n色 O\n系 O\n开 O\n业 O\n花 O\n篮 O\n单 O\n个 O\n（ O\n2 O\n个 O\n起 O\n拍 O\n） O', role='assistant', function_call=None, tool_calls=None)


In [ ]:
result = completion.choices[0].message.content.split('\n')

In [ ]:
result

['粉 O',
 '金 O',
 '粉 O',
 '玫 O',
 '瑰 O',
 '香 O',
 '槟 O',
 '色 O',
 '系 O',
 '开 O',
 '业 O',
 '花 O',
 '篮 O',
 '单 O',
 '个 O',
 '（ O',
 '2 O',
 '个 O',
 '起 O',
 '拍 O',
 '） O']

In [ ]:
pred = [re.split(' ', line) for line in result]
# pred = [line[1] for line in pred]

In [ ]:
pred

[['粉', 'O'],
 ['金', 'O'],
 ['粉', 'O'],
 ['玫', 'O'],
 ['瑰', 'O'],
 ['香', 'O'],
 ['槟', 'O'],
 ['色', 'O'],
 ['系', 'O'],
 ['开', 'O'],
 ['业', 'O'],
 ['花', 'O'],
 ['篮', 'O'],
 ['单', 'O'],
 ['个', 'O'],
 ['（', 'O'],
 ['2', 'O'],
 ['个', 'O'],
 ['起', 'O'],
 ['拍', 'O'],
 ['）', 'O']]

In [ ]:
query_tags[0]

In [ ]:
[tag2id[tag] for tag in query_tags[0]]

[42, 42, 42, 119, 113, 42, 42, 42, 42, 42, 42, 42, 42]

In [ ]:
def check_format(result):
    for i in range(len(result)):
        if '' in result[i]:
            result.pop()
            result.pop()
            break
    return result

## Evaluation


In [ ]:
from sklearn.metrics import f1_score
import statistics
def compute_metrics(preds, labels):
    """
    This function computes the confusion matrix for the given prediction and label
    """

    return f1_score(labels,preds,average='micro')

def average_f1(f1_list):
    return statistics.mean(f1_list)

In [ ]:
def check_key(dict, predictions):
    """
    this function convert prediction to its id
    by checking if the tag exists; otherwise assign to
    O-class

    dict: the tag2id dictionary
    predictions: the list of prediciton
    """
    for i in range(len(predictions)):
        pred = predictions[i]
        # print(pred)
        if pred in dict.keys():
            # print(pred, ' in map')
            predictions[i] = dict[pred]
        else:
            predictions[i] = dict['O']
    return predictions



In [ ]:
def validate_dimension(preds, labels):
    '''
    This function validate size of two input to check if they
    have the same size; otherwise, it drim the list from the back.

    '''
    if len(preds) == len(labels):
        return preds, labels
    else:
        len_p = len(preds)
        len_l = len(labels)
        if len_p > len_l:
# if length of prediction is longer than the label, remove those extra elements
            preds = preds[:len_l]
            return preds, labels
        else:
            labels = labels[:len_p]
            return preds, labels

In [ ]:
def validate_dimension(preds, labels):
    '''
    This function validate size of two input to check if they
    have the same size; otherwise, it drim the list from the back.

    '''
    if len(preds) == len(labels):
        return preds, labels
    else:
        len_p = len(preds)
        len_l = len(labels)
        if len_p > len_l:
# if length of prediction is longer than the label, remove those extra elements
            preds = preds[:len_l]
            return preds, labels
        else:
            labels = labels[:len_p]
            return preds, labels

## One Shot

In [ ]:
query_texts = test_texts[:8000][:]
query_tags = test_tags[:8000][:]

In [ ]:
f1_scores = []

In [ ]:
# id list which record id of those prediction f1 scores lower than 0.4
# pred list will record the prediction from LLM
id_list = []
pred_list = []
low_f1_list = []

In [ ]:
f1_scores = []

for i in range(len(query_texts)):
    if(i%5==0):
        print(i)
    query = query_texts[i]
    query = ' '.join(query)
    true_label = query_tags[i]


    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
        ]
        )


    result = completion.choices[0].message.content.split('\n')

    pred = [re.split(' ', line) for line in result]
    pred = [line[1] for line in pred]
    pred = check_key(tag2id,pred)

    # convert tag to id for f1 score calculation
    true_label = [int(tag2id[tag]) for tag in true_label]

    # validate input
    pred, true_label = validate_dimension(pred, true_label)

    f1_score = compute_metrics(true_label, pred)
    if f1 < 0.4:
        # record those low f1_score prediction for error analysis
        id_list.append(i)
        pred_list.append(word_pred)
        low_f1_list.append(f1)
    f1_scores.append(f1_score)



In [ ]:
print("One shot :",average_f1(f1_scores))

In [ ]:
low_f1_list

## Five Shot

In [ ]:
five_shot_f1_scores = []

In [ ]:
five_shot_query_texts = test_texts[:8000][:]
five_shot_query_tags = test_tags[:8000][:]

In [ ]:
num_query_tags = len(unique_tags)
text_mapping = {}
assigned = [0]*num_query_tags


In [ ]:
# sampling five instance for each class
test_tags = query_tags[:8000]
for i in range(len(test_tags)):
    tag = test_tags[i]
    for j in range(len(tag)):
        if tag[j] != 'O':
            # the index corresponds to the id of the tag
            id = tag2id[tag[j]]
            word = query_texts[i][j]
            if assigned[id] < 5:
                if assigned[id] == 0:
                    word_list = [word]
                    text_mapping[tag[j]] = word_list
                    assigned[id] += 1
                else:
                    word_list = text_mapping[tag[j]]
                    if word not in word_list:
                        word_list.append(word)
                        assigned[id] += 1
                    text_mapping[tag[j]] = word_list

In [ ]:
text_mapping

In [ ]:
# id list which record id of those prediction f1 scores lower than 0.4
# pred list will record the prediction from LLM
id_list = []
pred_list = []
low_f1_list = []

In [ ]:
query = test_text
query = ' '.join(query)


In [ ]:
query

'秋 冬 季 新 款 保 暖 棉 拖 鞋 居 家 室 内 踩 屎 感 静 音 厚 底 简 约 防 滑 情 侣 拖 鞋'

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
    ]
    )

In [ ]:
result = completion.choices[0].message.content.split('\n')

In [ ]:
result

['秋 O',
 '冬 O',
 '季 O',
 '新 O',
 '款 O',
 '保 O',
 '暖 O',
 '棉 O',
 '拖 B-鞋垫材质',
 '鞋 I-鞋垫材质',
 '居 O',
 '家 O',
 '室 O',
 '内 O',
 '踩 O',
 '屎 O',
 '感 O',
 '静 O',
 '音 O',
 '厚 B-厚度',
 '底 O',
 '简 O',
 '约 O',
 '防 O',
 '滑 O',
 '情 O',
 '侣 O',
 '拖 O',
 '鞋 O']

In [ ]:
pred = [re.split('', line) for line in result]

In [ ]:
pred

[['秋', 'O'],
 ['冬', 'O'],
 ['季', 'O'],
 ['新', 'O'],
 ['款', 'O'],
 ['保', 'O'],
 ['暖', 'O'],
 ['棉', 'B-面料材质'],
 ['拖', 'I-面料材质'],
 ['鞋', 'I-面料材质'],
 ['居', 'O'],
 ['家', 'O'],
 ['室', 'O'],
 ['内', 'O'],
 ['踩', 'O'],
 ['屎', 'O'],
 ['感', 'O'],
 ['静', 'O'],
 ['音', 'O'],
 ['厚', 'O'],
 ['底', 'O'],
 ['简', 'O'],
 ['约', 'O'],
 ['防', 'O'],
 ['滑', 'O'],
 ['情', 'O'],
 ['侣', 'O'],
 ['拖', 'O'],
 ['鞋', 'O']]

In [ ]:
[line[0] for line in pred]

['秋 O',
 '冬 O',
 '季 O',
 '新 O',
 '款 O',
 '保 O',
 '暖 O',
 '棉 O',
 '拖 B-鞋垫材质',
 '鞋 I-鞋垫材质',
 '居 O',
 '家 O',
 '室 O',
 '内 O',
 '踩 O',
 '屎 O',
 '感 O',
 '静 O',
 '音 O',
 '厚 B-厚度',
 '底 O',
 '简 O',
 '约 O',
 '防 O',
 '滑 O',
 '情 O',
 '侣 O',
 '拖 O',
 '鞋 O']

In [ ]:


for i in range(len(five_shot_query_texts)):
    if(i%10==0):
        print(i)
    query = five_shot_query_texts[i]
    query = ' '.join(query)
    true_label = query_tags[i]


    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
        ]
        )


    result = completion.choices[0].message.content.split('\n')

    pred = [re.split(' ', line) for line in result]

    print(pred)
    pred = [line[1] for line in pred]
    pred = check_key(tag2id,pred)

    # convert tag to id for f1 score calculation
    true_label = [int(tag2id[tag]) for tag in true_label]

    # validate input
    pred, true_label = validate_dimension(pred, true_label)
    f1 = compute_metrics(true_label, pred)
    if f1 < 0.4:
        # record those low f1_score prediction for error analysis
        id_list.append(i)
        pred_list.append(word_pred)
        low_f1_list.append(f1)
    five_shot_f1_scores.append(f1)

0
[['秋', 'O'], ['冬', 'O'], ['季', 'O'], ['新', 'O'], ['款', 'O'], ['保', 'B-功能功效'], ['暖', 'I-功能功效'], ['棉', 'B-面料材质'], ['拖', 'O'], ['鞋', 'O'], ['居', 'O'], ['家', 'O'], ['室', 'O'], ['内', 'O'], ['踩', 'O'], ['屎', 'O'], ['感', 'O'], ['静', 'O'], ['音', 'O'], ['厚', 'B-厚度'], ['底', 'O'], ['简', 'O'], ['约', 'O'], ['防', 'O'], ['滑', 'O'], ['情', 'O'], ['侣', 'O'], ['拖', 'O'], ['鞋', 'O']]
[['粉', 'B-颜色'], ['金', 'I-颜色'], ['粉', 'I-颜色'], ['玫', 'I-颜色'], ['瑰', 'I-颜色'], ['香', 'B-分类'], ['槟', 'I-分类'], ['色', 'I-分类'], ['系', 'O'], ['开', 'O'], ['业', 'O'], ['花', 'B-适用对象'], ['篮', 'I-适用对象'], ['单', 'O'], ['个', 'O'], ['（', 'O'], ['2', 'O'], ['个', 'O'], ['起', 'O'], ['拍', 'O'], ['）', 'O']]
[['维', 'B-适用对象'], ['达', 'I-适用对象'], ['v', 'O'], ['i', 'O'], ['n', 'O'], ['d', 'O'], ['a', 'O'], ['洁', 'B-外观'], ['肤', 'I-外观'], ['去', 'O'], ['油', 'O'], ['冰', 'B-功能功效'], ['爽', 'I-功能功效'], ['湿', 'B-适用对象'], ['巾', 'I-适用对象'], ['3', 'O'], ['包', 'O'], ['/', 'O'], ['包', 'O'], ['件', 'O']]
[['芬', 'B-商品特色'], ['达', 'I-商品特色'], ['汽', 'I-商品特色'], ['水', 'I-商品特色']

IndexError: list index out of range

In [ ]:
print("Five shot :",average_f1(five_shot_f1_scores))

0.668141053530025

In [ ]:
low_f1_list